# Test Data Drawing
---
Note that executing this script will overwrite the current test.csv file. This is by design. After testing models on the current data file, I will throw it out and draw another file.

In [1]:
import pandas as pd
import numpy as np

import praw
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [2]:
reddit = praw.Reddit(client_id = 'REDACTED',
                     client_secret = 'REDACTED',
                     user_agent = 'reddit_api', 
                     username = 'REDACTED',
                     password = 'REDACTED') 

In [3]:
subreddit = reddit.subreddit('Conservative')  
altsubreddit = reddit.subreddit('Progressive')

In [4]:
subreddit_new = subreddit.new(limit = 400)
subreddit_new_2 = altsubreddit.new(limit = 400)

In [5]:
topics_dict = { "title":[],
                "score":[],
                "id":[],
                "url":[], 
                "comms_num": [],
                "created": [],
                "body":[]}

#Use a for loop to take the posts gathered by the wrapper and place them into a dictionary.

for item in subreddit_new:
    topics_dict["title"].append(item.title)
    topics_dict["score"].append(item.score)
    topics_dict["id"].append(item.id)
    topics_dict["url"].append(item.url)
    topics_dict["comms_num"].append(item.num_comments)
    topics_dict["created"].append(item.created)
    topics_dict["body"].append(item.selftext)

In [6]:
topics_dict2 = { "title":[],
                "score":[],
                "id":[],
                "url":[], 
                "comms_num": [],
                "created": [],
                "body":[]}

#Use a for loop to take the posts gathered by the wrapper and place them into a dictionary.

for item in subreddit_new_2:
    topics_dict2["title"].append(item.title)
    topics_dict2["score"].append(item.score)
    topics_dict2["id"].append(item.id)
    topics_dict2["url"].append(item.url)
    topics_dict2["comms_num"].append(item.num_comments)
    topics_dict2["created"].append(item.created)
    topics_dict2["body"].append(item.selftext)

In [7]:
df = pd.DataFrame(topics_dict)
df2 = pd.DataFrame(topics_dict2)

In [8]:
df['subreddit'] = "Conservative"
df2['subreddit'] = 'Progressive'

In [9]:
df_final = pd.concat([df, df2])

In [10]:
#Praw commentforest:
comments_for_df = []

for ids in df_final.id:
    submission = reddit.submission(id = ids)
    entry = {}
    
    entry['id'] = ids
    comms = []
    submission.comments.replace_more(limit=1)
    for comment in submission.comments.list():
        comms.append(comment.body)
    entry['comments'] = comms
    comments_for_df.append(entry)

In [11]:
comments = pd.DataFrame(comments_for_df)

In [12]:
reddit = df_final.merge(comments[['id', 'comments']],how = 'outer', left_on = 'id', right_on = 'id')

In [13]:
reddit.head()

,title,score,id,url,comms_num,created,body,subreddit,comments
0,Vigano Responds to Pope's Nod to Homosexual Ci...,1,jg3su9,https://www.churchmilitant.com/news/article/de...,0,1.603415e+09,,Conservative,[]
1,"The Atlantic Council, a prominent political th...",5,jfzq6y,https://twitter.com/JackPosobiec/status/131926...,1,1.603402e+09,,Conservative,[This is important because The Atlantic Counci...
2,"President Trump Leaks Raw Footage From ""60 Min...",6,jg3ng6,https://www.realclearpolitics.com/video/2020/1...,0,1.603415e+09,,Conservative,[]
3,NPR: We’re Not Covering Biden Laptop Scandal B...,3,jg3mb6,https://www.dailywire.com/news/npr-were-not-co...,2,1.603415e+09,,Conservative,[But Trump and colluding with Russia ( even th...
4,"Ex Partner FLIPS ON Joe Biden, Biden Signed Of...",11,jg0bt9,https://youtu.be/UBY4ig7SPm8,1,1.603405e+09,,Conservative,[]


In [14]:
#This quickly enters NaN values and collapses my comments into one list.
reddit.to_csv('Data/tests/test.csv', index = False)

In [15]:
#Read it back in to continue!
reddit = pd.read_csv('Data/tests/test.csv')

In [16]:
reddit['text'] = reddit.apply(lambda row: row.title + row.body + row.comments if not pd.isnull(row.body) and 
                              not pd.isnull(row.comments) else row.title + row.comments if not 
                              pd.isnull(row.comments) else row.title + row.body if not pd.isnull(row.body) 
                              else row.title, axis = 1)

In [17]:
reddit['sites'] = reddit.url.apply(lambda x: x.replace('https://www.', '').replace('https://', '').replace('http://', '').split('/')[0])

In [18]:
reddit.head()

,title,score,id,url,comms_num,created,body,subreddit,comments,text,sites
0,Vigano Responds to Pope's Nod to Homosexual Ci...,1,jg3su9,https://www.churchmilitant.com/news/article/de...,0,1.603415e+09,NaN,Conservative,[],Vigano Responds to Pope's Nod to Homosexual Ci...,churchmilitant.com
1,"The Atlantic Council, a prominent political th...",5,jfzq6y,https://twitter.com/JackPosobiec/status/131926...,1,1.603402e+09,NaN,Conservative,['This is important because The Atlantic Counc...,"The Atlantic Council, a prominent political th...",twitter.com
2,"President Trump Leaks Raw Footage From ""60 Min...",6,jg3ng6,https://www.realclearpolitics.com/video/2020/1...,0,1.603415e+09,NaN,Conservative,[],"President Trump Leaks Raw Footage From ""60 Min...",realclearpolitics.com
3,NPR: We’re Not Covering Biden Laptop Scandal B...,3,jg3mb6,https://www.dailywire.com/news/npr-were-not-co...,2,1.603415e+09,NaN,Conservative,['But Trump and colluding with Russia ( even t...,NPR: We’re Not Covering Biden Laptop Scandal B...,dailywire.com
4,"Ex Partner FLIPS ON Joe Biden, Biden Signed Of...",11,jg0bt9,https://youtu.be/UBY4ig7SPm8,1,1.603405e+09,NaN,Conservative,[],"Ex Partner FLIPS ON Joe Biden, Biden Signed Of...",youtu.be


In [19]:
reddit.to_csv('Data/tests/test.csv', index = False)

In [20]:
test = pd.read_csv('Data/tests/test.csv')
test.head()

,title,score,id,url,comms_num,created,body,subreddit,comments,text,sites
0,Vigano Responds to Pope's Nod to Homosexual Ci...,1,jg3su9,https://www.churchmilitant.com/news/article/de...,0,1.603415e+09,NaN,Conservative,[],Vigano Responds to Pope's Nod to Homosexual Ci...,churchmilitant.com
1,"The Atlantic Council, a prominent political th...",5,jfzq6y,https://twitter.com/JackPosobiec/status/131926...,1,1.603402e+09,NaN,Conservative,['This is important because The Atlantic Counc...,"The Atlantic Council, a prominent political th...",twitter.com
2,"President Trump Leaks Raw Footage From ""60 Min...",6,jg3ng6,https://www.realclearpolitics.com/video/2020/1...,0,1.603415e+09,NaN,Conservative,[],"President Trump Leaks Raw Footage From ""60 Min...",realclearpolitics.com
3,NPR: We’re Not Covering Biden Laptop Scandal B...,3,jg3mb6,https://www.dailywire.com/news/npr-were-not-co...,2,1.603415e+09,NaN,Conservative,['But Trump and colluding with Russia ( even t...,NPR: We’re Not Covering Biden Laptop Scandal B...,dailywire.com
4,"Ex Partner FLIPS ON Joe Biden, Biden Signed Of...",11,jg0bt9,https://youtu.be/UBY4ig7SPm8,1,1.603405e+09,NaN,Conservative,[],"Ex Partner FLIPS ON Joe Biden, Biden Signed Of...",youtu.be
